In [ ]:
!pip install google.colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!lsb_release -a

In [ ]:
!nvidia-smi

In [ ]:
!nvcc -V

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.device(0))

In [ ]:
# 註: 如果已經有安裝 PyTorch，就不用再安裝了
# !pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%cd /content/drive/My Drive/colab_test/

In [ ]:
!git clone https://github.com/Morizeyao/GPT2-Chinese.git
%cd GPT2-Chinese
!pip install -r requirements.txt
%cd ..

In [ ]:
%cd GPT2-Chinese